In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt
from keras import Sequential
from keras import layers
import os
from keras.preprocessing import image

In [93]:
train_ds = keras.preprocessing.image_dataset_from_directory(
  './sampled_classes/train/', 
  labels='inferred',
  label_mode='categorical', 
  seed=123, 
  batch_size=32, 
  image_size=(224, 224)
)
  
val_ds = keras.preprocessing.image_dataset_from_directory(
  './sampled_classes/validation/', 
  labels='inferred',
  label_mode='categorical',
  seed=123,
  image_size=(224, 224),
  batch_size=32)



Found 2625 files belonging to 7 classes.
Found 875 files belonging to 7 classes.


In [47]:
from keras.applications.vgg19 import VGG19
from keras.applications.vgg19 import preprocess_input

model_t = VGG19(
    include_top=False,
    weights="imagenet",
    input_shape=(224, 224, 3),
)
model_t.trainable = False

train_ds = train_ds.map(lambda x,y : (preprocess_input(x), y))
val_ds = val_ds.map(lambda x,y : (preprocess_input(x), y))

In [61]:
model_t.input

<KerasTensor: shape=(None, 224, 224, 3) dtype=float32 (created by layer 'input_4')>

In [45]:
# plt.figure(figsize=(10, 10))
# for i, (images, label) in enumerate(train_ds.take(9)):
#     ax = plt.subplot(3, 3, i + 1)
#     plt.imshow(images[0]/225)
    

In [83]:
new_model = Sequential([
    layers.RandomRotation(factor=1, input_shape=(224, 224, 3)),
    model_t,
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dense(7, activation='softmax')
])

In [76]:
inputs = model_t.input
x = layers.RandomRotation(factor=1)(inputs)
# x = preprocess_input(x)
x = model_t(x, training=False)
global_average_layer = layers.GlobalAveragePooling2D()
x=global_average_layer(x)
x = layers.Dropout(0.2)(x)
outputs=layers.Dense(7, activation='softmax')(x)
model = keras.Model(inputs, outputs)

In [77]:
model.summary()

Model: "model_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_4 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
random_rotation_4 (RandomRot (None, 224, 224, 3)       0         
_________________________________________________________________
tf.__operators__.getitem_2 ( (None, 224, 224, 3)       0         
_________________________________________________________________
tf.nn.bias_add_2 (TFOpLambda (None, 224, 224, 3)       0         
_________________________________________________________________
vgg19 (Functional)           (None, 7, 7, 512)         20024384  
_________________________________________________________________
global_average_pooling2d_2 ( (None, 512)               0         
_________________________________________________________________
dropout_2 (Dropout)          (None, 512)               0   

In [78]:
model.compile(
    optimizer='adam', 
    loss=keras.metrics.CategoricalCrossentropy(),
    metrics=['categorical_accuracy']) 

In [79]:
from keras import callbacks
earlystopping = callbacks.EarlyStopping(monitor ="val_categorical_accuracy", 
                                        mode ="max", 
                                        patience = 5, 
                                        restore_best_weights = True,
                                        verbose=2)

In [88]:
for images, labels in train_ds.take(1):
    train_X = images
    train_y = labels

for images, labels in val_ds.take(1):
    val_X = images
    val_y = labels


In [90]:
history = model.fit(train_ds,
    validation_data=val_ds,
    epochs=20,
    callbacks=earlystopping
)

Epoch 1/20


ValueError: in user code:

    /Users/wvdvijve/miniconda3/envs/tf/lib/python3.8/site-packages/keras/engine/training.py:853 train_function  *
        return step_function(self, iterator)
    /Users/wvdvijve/miniconda3/envs/tf/lib/python3.8/site-packages/keras/engine/training.py:842 step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    /Users/wvdvijve/miniconda3/envs/tf/lib/python3.8/site-packages/tensorflow/python/distribute/distribute_lib.py:1286 run
        return self._extended.call_for_each_replica(fn, args=args, kwargs=kwargs)
    /Users/wvdvijve/miniconda3/envs/tf/lib/python3.8/site-packages/tensorflow/python/distribute/distribute_lib.py:2849 call_for_each_replica
        return self._call_for_each_replica(fn, args, kwargs)
    /Users/wvdvijve/miniconda3/envs/tf/lib/python3.8/site-packages/tensorflow/python/distribute/distribute_lib.py:3632 _call_for_each_replica
        return fn(*args, **kwargs)
    /Users/wvdvijve/miniconda3/envs/tf/lib/p

# Poging 2

In [91]:
# Import your chosen model!
from tensorflow.keras.applications.mobilenet_v2 import MobileNetV2

# Make a model object. 
# Make sure you exclude the top part. set the input shape of the model to 224x224 pixels, with 3 color channels.
model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(224,224,3))

# Freeze the imported layers so they cannot be retrained.
for layer in model.layers:
    layer.trainable = False

9420800/9406464 [==============================] - 1s 0us/step


In [98]:
from keras import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Dropout
new_model = Sequential()
new_model.add(model)
new_model.add(Flatten())
new_model.add(Dense(64, activation='relu'))
new_model.add(Dropout(0.5))
new_model.add(Dense(7, activation='sigmoid'))

# Summarize.

In [94]:
from keras.applications.mobilenet_v2 import preprocess_input
train_ds = train_ds.map(lambda x,y : (preprocess_input(x), y))
val_ds = val_ds.map(lambda x,y : (preprocess_input(x), y))

In [99]:
new_model.compile(
    optimizer='adam', 
    loss=keras.metrics.CategoricalCrossentropy(),
    metrics=['categorical_accuracy']) 

In [100]:
history = new_model.fit(train_ds,
    validation_data=val_ds,
    epochs=2,
    callbacks=earlystopping
)

Epoch 1/2


ValueError: in user code:

    /Users/wvdvijve/miniconda3/envs/tf/lib/python3.8/site-packages/keras/engine/training.py:853 train_function  *
        return step_function(self, iterator)
    /Users/wvdvijve/miniconda3/envs/tf/lib/python3.8/site-packages/keras/engine/training.py:842 step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    /Users/wvdvijve/miniconda3/envs/tf/lib/python3.8/site-packages/tensorflow/python/distribute/distribute_lib.py:1286 run
        return self._extended.call_for_each_replica(fn, args=args, kwargs=kwargs)
    /Users/wvdvijve/miniconda3/envs/tf/lib/python3.8/site-packages/tensorflow/python/distribute/distribute_lib.py:2849 call_for_each_replica
        return self._call_for_each_replica(fn, args, kwargs)
    /Users/wvdvijve/miniconda3/envs/tf/lib/python3.8/site-packages/tensorflow/python/distribute/distribute_lib.py:3632 _call_for_each_replica
        return fn(*args, **kwargs)
    /Users/wvdvijve/miniconda3/envs/tf/lib/p